In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("../data/codigos_cambiados.xlsx")

In [4]:
import numpy as np
import pandas as pd

# Define a dictionary for the descriptions and their corresponding values
descriptions_mapping = {
    'Adquisición neta de activos financieros': 'ANAF',
    'Emisión neta de pasivos': 'ENP'
}

# Function to assign values based on description
def assign_description_value(row):
    return descriptions_mapping.get(row['Descripción'], np.nan)

# Apply the function to create a new column
df['anaf_enp'] = df.apply(assign_description_value, axis=1)

# Count hierarchy levels in 'Código BDP'
df['n_jerarquía'] = df['Código BDP'].str.count('\.') +  1

# Create 'cod_anaf_mas_cercano' and 'n_jerarquía_anaf_mas_cercano' columns using shift
df['cod_anaf_mas_cercano'] = df['Código BDP'].where(df['anaf_enp'] == 'ANAF').ffill().fillna(np.nan)
df['n_jerarquía_anaf_mas_cercano'] = df['n_jerarquía'].where(df['anaf_enp'] == 'ANAF').ffill().fillna(np.nan)

# Function to determine dependency
def calculate_dependency(row):
    return (pd.notnull(row['n_jerarquía_anaf_mas_cercano']) and
            row['n_jerarquía'] > row['n_jerarquía_anaf_mas_cercano'] and
            row['Código BDP'].startswith(row['cod_anaf_mas_cercano']))

# Apply the function to create 'depende_anaf_enp' column
df['depende_anaf_enp'] = df.apply(calculate_dependency, axis=1)

# Function to calculate the own code
def calculate_own_code(row):
    if pd.notnull(row['cod_anaf_mas_cercano']):
        index_anaf = row['Código BDP'].find(row['cod_anaf_mas_cercano'])
        if index_anaf != -1:
            return row['Código BDP'][index_anaf + len(row['cod_anaf_mas_cercano']):]
    return np.nan

# Apply the function to create 'codigo_propio' column
df['codigo_propio'] = df.apply(calculate_own_code, axis=1)

# Function to find the first occurrence of 'ANAF' or 'ENP'
def find_first_anaf_or_enp(row):
    index_current = row.name
    while index_current >=  0:
        value_previous = df.at[index_current, 'anaf_enp']
        if pd.notnull(value_previous):
            return value_previous
        index_current -=  1
    return np.nan

# Apply the function to create 'primero_anaf_o_enp' column
df['primero_anaf_o_enp'] = df.apply(find_first_anaf_or_enp, axis=1)

# Function to create a new code
def create_new_code(row):
    if row['depende_anaf_enp']:
        return f"{row['cod_anaf_mas_cercano']}.{row['primero_anaf_o_enp']}{row['codigo_propio']}"
    elif pd.notnull(row['anaf_enp']):
        return f"{row['Código BDP']}.{row['anaf_enp']}"
    else:
        return row['Código BDP']

# Apply the function to update 'Código BDP' column
df['Código BDP'] = df.apply(create_new_code, axis=1)

# Select only required columns
df = df[["SDMX", "Código BDP", "Descripción"]]

# Display the DataFrame
df


,SDMX,Código BDP,Descripción
0,Q.N.AR.W1.S1.S1.T.N.FA._T.F._Z.USD._T._X.N,3,Cuenta financiera
1,Q.N.AR.W1.S1.S1.T.N.FA.D.F._Z.USD._T._X.N,3.1,Inversión directa
2,Q.N.AR.W1.S1.S1.T.A.FA.D.F._Z.USD._T._X.N,3.1.ANAF,Adquisición neta de activos financieros
3,Q.N.AR.W1.S1.S1.T.A.FA.D.F5._Z.USD._T._X.N,3.1.ANAF.1,Participaciones de capital y en fondos de inve...
4,Q.N.AR.W1.S1.S1.T.A.FA.D.F5A._Z.USD._T._X.N,3.1.ANAF.1.1,Participaciones de capital distintas de reinve...
...,...,...,...
97,Q.N.AR.W1.S121.S1.T.A.FA.R.F._Z.USD.X1._X.N,3.5,Activos de reserva
98,Q.N.AR.W1.S121.S1.T.A.FA.R.F11._Z.USD.XAU.M.N,3.5.1,Oro monetario
99,Q.N.AR.W1.S121.S1N.T.A.FA.R.F12.T.USD.XDR.M.N,3.5.2,Derechos especiales de giro
100,Q.N.AR.1C.S121.S121.T.A.FA.R.FK._Z.USD.XDR.M.N,3.5.3,Posición de reserva en el FMI
